<h1 style="font-size:50px; text-align:center">DCO Detections</h1>

In [1]:
import h5py as h5
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
import seaborn as sns
import legwork
import os
import sys
sys.path.append("../src/")
from variations import variations
from galaxy import simulate_mw

In [2]:
%config InlineBackend.figure_format = 'retina'
plt.rc('font', family='serif')
fs = 20

params = {'legend.fontsize': fs,
         'axes.labelsize': fs,
         'xtick.labelsize':0.7*fs,
         'ytick.labelsize':0.7*fs}
plt.rcParams.update(params)

In [3]:
dco_colours = {"BHBH": plt.get_cmap("plasma")(0.2), 
               "BHNS": plt.get_cmap("plasma")(0.5),
               "NSNS": plt.get_cmap("plasma")(0.8)}

dco_types = ["BHBH", "BHNS", "NSNS"]
data_path = "../data/"
sim_folder = data_path + "simulation/"
simple_sim_folder = data_path + "simple_mw_simulation/"

total_DCOs_in_MW = np.load(data_path + "total_DCO_in_MW.npy")
total_DCOs_in_simple_MW = np.load(data_path + "total_DCO_in_simple_MW.npy")

total_DCOs_in_simple_MW_new = np.zeros_like(total_DCOs_in_MW)
total_DCOs_in_simple_MW_new[:, 0] = total_DCOs_in_simple_MW[0]
total_DCOs_in_simple_MW_new[:, 5] = total_DCOs_in_simple_MW[1]
total_DCOs_in_simple_MW = total_DCOs_in_simple_MW_new

In [75]:
simple_mw_fiducial = detections_4yr[:, 0].mean(axis=1) / total_DCOs_in_MW[:, 0] * total_DCOs_in_simple_MW[0, :]
simple_mw_katie = detections_4yr[:, 5].mean(axis=1) / total_DCOs_in_MW[:, 5] * total_DCOs_in_simple_MW[1, :]

In [78]:
print(simple_mw_fiducial.round().astype(int))
print(simple_mw_katie.round().astype(int))

[52 22 11]
[80 74 22]


# Calculate number of detections

In [4]:
def get_detections(dco_types=dco_types, variation_list=range(len(variations)), folder=sim_folder, totals=total_DCOs_in_MW, t_obs=4*u.yr):
    # set up arrays for return
    detections = np.zeros(shape=(len(dco_types), len(variations), 2500))

    for d in range(len(dco_types)):
        for v in variation_list:
            # open the proper output file
            fname = folder + "{}_{}_all.h5".format(dco_types[d], variations[v]["file"])
            if os.path.isfile(fname):
                with h5.File(fname, "r") as f:
                    parameter = "n_detect"
                    if v in [17, 19]:
                        parameter = "n_ten_year"
                    n_detect = f["simulation"].attrs[parameter].astype(np.int)
                    total_mw_weight = f["simulation"].attrs["total_MW_weight"]
                    full_data = f["simulation"][...].squeeze()

                # go through the file and normalise the detections for all binaries with SNR > x
                cursor = 0
                detections_per_MW = np.zeros(len(n_detect))
                
                checks = np.zeros(len(n_detect))
                for i in range(len(n_detect)):
                    snr = full_data["snr"][cursor:cursor + n_detect[i]]
                    weights = full_data["weight"][cursor:cursor + n_detect[i]]
                    checks = np.sum(weights)**2 / np.sum(weights**2)
                    detections[d][v][i] = np.sum(weights[snr * np.sqrt(t_obs / (4 * u.yr)) > 7]) / total_mw_weight[i] * totals[d][v]
                    cursor += n_detect[i]

            # set to zero if no file exists (simulation will have crashed)
            else:
                detections[d][v] = 0
                
            print(d, v, detections[d][v].mean())
    
    return detections

In [56]:
detections_simple = get_detections(folder=simple_sim_folder + "simple_mw_", variation_list=[0, 5], totals=total_DCOs_in_simple_MW)

0 0 52.3751446651769
0 5 96.72893783753781
1 0 24.93725218656574
1 5 101.39867208989773
2 0 17.374324879667842
2 5 43.457652745656496


In [63]:
detections_simple[:, 5].mean(axis=1).round().astype(int)

array([ 97, 101,  43])

In [80]:
(detections_4yr[:, 11].mean(axis=1) / total_DCOs_in_MW[:, 11]) / (detections_4yr[:, 0].mean(axis=1) / total_DCOs_in_MW[:, 0])

array([0.75827354, 0.79552324, 1.00616999])

In [81]:
total_DCOs_in_MW[:, 11] / total_DCOs_in_MW[:, 0]

array([0.89879758, 2.08820566, 0.83680915])

In [83]:
(detections_4yr[:, 18].mean(axis=1) / total_DCOs_in_MW[:, 18]) / (detections_4yr[:, 0].mean(axis=1) / total_DCOs_in_MW[:, 0])

array([0.70287485, 0.75264659, 1.19052106])

In [82]:
total_DCOs_in_MW[:, 18] / total_DCOs_in_MW[:, 0]

array([1.45605119, 1.06884256, 0.93392604])

array([1.09217310e-04, 8.73358274e-05, 5.16216468e-05])

In [5]:
detections_4yr = get_detections()

0 0 74.0329532213235
0 1 68.8149290199246
0 2 46.99094453972454
0 3 46.91890587905573
0 4 69.26099797363257
0 5 154.3290318193001
0 6 27.93464327369012
0 7 58.2729097140852
0 8 67.6154535109307
0 9 26.70402162619144
0 10 151.51430066855596
0 11 50.389973861134926
0 12 96.21717718968601
0 13 58.252718934233485
0 14 75.33006261623602
0 15 82.66221097394889
0 16 91.84220949516094
0 17 86.1838335035012
0 18 75.67522813889646
0 19 0.0
1 0 42.35845607368241
1 1 22.404356730637712
1 2 8.255220459339595
1 3 7.426608930563379
1 4 7.25588173138878
1 5 148.35334375131637
1 6 2.1124608250568686
1 7 21.754622932183523
1 8 37.96281102288457
1 9 16.309998128093063
1 10 56.81392835161141
1 11 70.23141789580879
1 12 30.105679754325177
1 13 51.90679829236058
1 14 43.355132494925826
1 15 86.57157445373693
1 16 142.87203884283383
1 17 96.10273007453586
1 18 34.0127559258105
1 19 14.95590921147829
2 0 7.9437775048188515
2 1 2.872357175852811
2 2 3.941399857096911
2 3 12.698251280785508
2 4 0.22172825228413

<ipython-input-4-94c9bded0a7e>:26: RuntimeWarning: invalid value encountered in double_scalars
  checks = np.sum(weights)**2 / np.sum(weights**2)


2 16 33.57742069286192
2 17 9.731617839656844
2 18 8.78340758800099
2 19 9.608981019431885


In [6]:
detections_10yr = get_detections(dco_types, t_obs=10*u.yr)

In [7]:
detections_only_fractions = get_detections(dco_types, only_frac=True)

In [7]:
np.save("../data/detections_4yr", detections_4yr)
np.save("../data/detections_10yr", detections_10yr)
np.save("../data/detections_only_fractions", detections_only_fractions)

NameError: name 'detections_10yr' is not defined

# What if we assumed everything was circular?

In [55]:
fid_sources = [None, None, None]
total_mw_weights = [None, None, None]
for i in range(len(dco_types)):
    with h5.File(sim_folder + "{}_massTransferEfficiencyFixed_0_5_all.h5".format(dco_types[i]), "r") as f:
        full_data = f["simulation"][...].squeeze()
        snr_mask = full_data["snr"] > 7
        
        data = full_data[snr_mask]
        
        total_mw_weights[i] = f["simulation"].attrs["total_MW_weight"].sum()
        
    fid_sources[i] = legwork.source.Source(m_1=data["m_1"] * u.Msun, m_2=data["m_2"] * u.Msun,
                                           dist=data["dist"] * u.kpc, ecc=data["e_LISA"],
                                           a=data["a_LISA"] * u.AU)
    fid_sources[i].weight = data["weight"]
    fid_sources[i].zams_mask = np.logical_and(data["m_1_ZAMS"] >= 20, data["m_1_ZAMS"] <= 120)

In [56]:
for i in range(len(dco_types)):
    snr_true = fid_sources[i].get_snr()
    new_source = fid_sources[i]
    new_source.ecc = np.zeros(new_source.n_sources)
    snr_circ = new_source.get_snr()
    
    full_mask = np.logical_and(fid_sources[i].zams_mask, snr_circ > 7)
    
    print(np.sum(new_source.weight[full_mask]) / total_mw_weights[i] * total_DCOs_in_MW[i][2])

27.72703093695445
6.985324518363619
0.06933292048296986
